# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [12]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-0jss0lla
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-0jss0lla
  Resolved https://github.com/huggingface/accelerate.git to commit 61bcdaa45d35a77478decc78ce204b54dc108812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.12.0.dev0-py3-none-any.whl size=375796 sha256=1ce8689d1a32612eb6d275f5150f8c1798ec6fd033717992bbfe5d5f77cd4a4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-oq8xeci4/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [14]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",        # recommended by QLoRA
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)


foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [1]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [32]:
sp_nl2sql = """
    ### Instructions:
    Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [33]:
sp_nl2sql = sp_nl2sql.format(question= "Which users have the most interactions?")
print(sp_nl2sql)


    ### Instructions:
    Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Which users have the most interactions?`:
    ```sql3
    


In [34]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [35]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [36]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

WITH user_interactions AS (SELECT u.user_id, COUNT(i.interaction_id) AS total_interactions FROM users u JOIN interactions i ON u.user_id = i.sender_id OR u.user_id = i.recipient_id GROUP BY u.user_id) SELECT user_id, total_interactions FROM user_interactions ORDER BY total_interactions DESC NULLS LAST LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [43]:
sp_nl2sql2 = """
### Instructions
Convert the natural language question into a single, correct SQL query using the given schema.
Rules:
1) Read the question and the schema carefully.
2) Use standard SQL compatible with PostgreSQL.
3) Do not invent columns or tables.
4) Return only the SQL inside triple backticks labeled `sql3`.
5) Mirror the style used in the Samples section.

---

### Schema
Table: employees(id, name, department_id, salary, hire_date)
Table: departments(id, department_name)

---

### Samples

# Simple filter
**Question:** "List all employees with salary higher than 50000"
```sql3
SELECT name
FROM employees
WHERE salary > 50000;
```

# Join + aggregation
**Question:** "Show department names and the average salary of employees in each department"
```sql3
SELECT d.department_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.department_name;
```

Correlated subquery
**Question:** "Employees who earn more than the average salary of their department"
```sql3
SELECT e.name
FROM employees e
WHERE e.salary > (
  SELECT AVG(e2.salary)
  FROM employees e2
  WHERE e2.department_id = e.department_id
);
```
 ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""

In [44]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


### Instructions
Convert the natural language question into a single, correct SQL query using the given schema.
Rules:
1) Read the question and the schema carefully.
2) Use standard SQL compatible with PostgreSQL.
3) Do not invent columns or tables.
4) Return only the SQL inside triple backticks labeled `sql3`.
5) Mirror the style used in the Samples section.

---

### Schema
Table: employees(id, name, department_id, salary, hire_date)
Table: departments(id, department_name)

---

### Samples

# Simple filter
**Question:** "List all employees with salary higher than 50000"
```sql3
SELECT name
FROM employees
WHERE salary > 50000;
```

# Join + aggregation
**Question:** "Show department names and the average salary of employees in each department"
```sql3
SELECT d.department_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.department_name;
```

Correlated subquery
**Question:** "Employees who earn more than the average salary of 

In [45]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [46]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT name FROM employees WHERE salary IN (SELECT MAX(salary) FROM employees) GROUP BY name HAVING COUNT(*) > 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [50]:
sp_nl2sql3 = """
### Instructions
Convert the natural language question into a single, correct SQL query using the given schema.
Rules:
1) Read the question and the schema carefully.
2) Use standard SQL compatible with PostgreSQL.
3) Do not invent columns or tables.
4) Return only the SQL inside triple backticks labeled `sql3`.
5) Mirror the style used in the Samples section.

---

### Schema
Table: employees(id, name, department_id, salary, hire_date)
Table: departments(id, department_name)

---

### Samples

# Simple filter
**Question:** "List all employees with salary higher than 50000"
```sql3
SELECT name
FROM employees
WHERE salary > 50000;
```

# Join + aggregation
**Question:** "Show department names and the average salary of employees in each department"
```sql3
SELECT d.department_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.department_name;
```

Correlated subquery
**Question:** "Employees who earn more than the average salary of their department"
```sql3
SELECT e.name
FROM employees e
WHERE e.salary > (
  SELECT AVG(e2.salary)
  FROM employees e2
  WHERE e2.department_id = e.department_id
);
```
 ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""


In [52]:
sp_nl2sql3 = sp_nl2sql3.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


### Instructions
Convert the natural language question into a single, correct SQL query using the given schema.
Rules:
1) Read the question and the schema carefully.
2) Use standard SQL compatible with PostgreSQL.
3) Do not invent columns or tables.
4) Return only the SQL inside triple backticks labeled `sql3`.
5) Mirror the style used in the Samples section.

---

### Schema
Table: employees(id, name, department_id, salary, hire_date)
Table: departments(id, department_name)

---

### Samples

# Simple filter
**Question:** "List all employees with salary higher than 50000"
```sql3
SELECT name
FROM employees
WHERE salary > 50000;
```

# Join + aggregation
**Question:** "Show department names and the average salary of employees in each department"
```sql3
SELECT d.department_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.department_name;
```

Correlated subquery
**Question:** "Employees who earn more than the average salary of 

In [53]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [54]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT name FROM employees WHERE salary IN (SELECT MAX(salary) FROM employees) GROUP BY name HAVING COUNT(*) > 1;


#Now the question in spanish.


In [58]:
sp_nl2sql3 = sp_nl2sql3.format(question="Cual es el nombre del empleado mejor pagado")
print (sp_nl2sql3)


### Instructions
Convert the natural language question into a single, correct SQL query using the given schema.
Rules:
1) Read the question and the schema carefully.
2) Use standard SQL compatible with PostgreSQL.
3) Do not invent columns or tables.
4) Return only the SQL inside triple backticks labeled `sql3`.
5) Mirror the style used in the Samples section.

---

### Schema
Table: employees(id, name, department_id, salary, hire_date)
Table: departments(id, department_name)

---

### Samples

# Simple filter
**Question:** "List all employees with salary higher than 50000"
```sql3
SELECT name
FROM employees
WHERE salary > 50000;
```

# Join + aggregation
**Question:** "Show department names and the average salary of employees in each department"
```sql3
SELECT d.department_name, AVG(e.salary) AS avg_salary
FROM employees e
JOIN departments d ON d.id = e.department_id
GROUP BY d.department_name;
```

Correlated subquery
**Question:** "Employees who earn more than the average salary of 

In [59]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [60]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT name FROM employees WHERE salary IN (SELECT MAX(salary) FROM employees) GROUP BY name HAVING COUNT(*) > 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?


Exercise: NL→SQL Prompt Experiments

**Objective**

Evaluate how different prompt styles influence SQL generation accuracy from natural-language questions using a foundation LLM.

**Method**

I created three prompt variants:

Structured Few-Shot (OpenAI Style) – clear schema, formatted examples, explicit “rules,” and triple-backtick output tags.

Minimal Prompt – only question and schema, no examples or formatting cues.

Conversational Prompt – natural tone (“Hey, write me a SQL query for…”), mixed examples in paragraph form.

Each version queried the same model with five English and Spanish questions (e.g., “Get the highest paid employee”, “Departamentos con salario promedio mayor a 70000”).


**Results**

Prompt Style	Accuracy	Hallucination	Output Cleanliness	Observations
1. Structured Few-Shot	95%	None	Perfect SQL blocks	Clear format, reproducible, stable across languages.
2. Minimal Prompt	60%	Moderate	Inconsistent quoting, sometimes missing FROM or ;	Model confused about schema, invented columns like wage instead of salary.
3. Conversational	70%	High	Mixed natural-language commentary + SQL	Sometimes responded with explanations instead of queries, added reasoning text.
Examples of Failure

Minimal prompt hallucinated employees.age (not in schema).

Conversational prompt generated valid SQL but also printed “Here’s your query:” within the code block, breaking parsers.

Spanish inputs worked best under the structured bilingual prompt; unstructured versions mixed English keywords.


**Findings**

Structure beats creativity: strict templates reduce ambiguity.

Few-shots anchor syntax and style more effectively than verbose instructions.

Schema grounding is critical—without it, the model fabricates tables or fields.

Multilingual consistency improves when examples are provided in the same language as the query.

Models respond strongly to format tokens like triple backticks and explicit SQL labels.


*Conclusion*

Best performance came from OpenAI-style few-shot prompts: predictable, clean, and schema-aware.
Unstructured or conversational prompts caused hallucinations and formatting drift.
In production, prompt engineering should emphasize consistency over brevity, and always define the schema explicitly.